c++为什么比python快，这个问题涉及到多方面。

2025-11-23 22:55:27 Sunday first time write this note.

## 从执行流程的区别，可以大致把语言实现分成三类：

1. 源代码经过 编译器 的到机器码，cpu执行机器码。 如c++。

2. 源代码经过 编译器 得到字节码，解释器 执行字节码。 如python。

3. 源代码经过 编译器 得到字节码，再经过虚拟机得到机器码（其中对于非热点的字节码会在虚拟机中直接经过解释器执行，而对于热点字节码才会经过虚拟机中的JIT变成机器码），cpu执行机器码。 如java。

其中涉及到的问题：
### 1. 解释器和虚拟机的区别：
- 解释器是执行字节码。
- 虚拟机是一个负责运行字节码的系统，它一般包含解释器和JIT，会对字节码区分执行，对于热点字节码会经过JIT转化为机器码，再由cpu执行机器码；而对于非热点字节码，会直接经过解释器执行。如下图所示

                      +-----------------------------+
                      |           虚拟机 VM         |
                      |   (解释器 Interpreter + JIT |
                      |       + GC + 其他组件)       |
                      +-----------------------------+
                                   |
                          字节码 Bytecode
                                   |
                 +-----------------+-----------------+
                 |                                   |
          非热点（冷代码）                       热点（热代码）
                 |                                   |
           +-----------+                     +------------------+
           | 解释器执行 |                     |   JIT 编译成机器码 |
           +-----------+                     +------------------+
                 |                                   |
         CPU 执行解释器逻辑                   CPU 直接执行机器码（最快）


### 2. 解释器是什么？
解释器可以是一段代码，比如python解释器叫cpython，是c写的，开源在https://github.com/python/cpython/tree/main。

其中，cpython代码中的重要文件：
1. **Python/ceval.c** - 主循环
它做的事情就是：持续读取字节码 → 根据 opcode 执行对应 C 函数 → 修改 Python 栈 → 跳下一条指令.
用代码描述就是
    
    ```
    for (;;) {
        opcode = *next_instr++;
        switch(opcode) {
            case LOAD_FAST:
                ...
            case BINARY_OP:
                ...
    ```
`
2. **Python/bytecodes.c** - 字节码指令定义（源文件）

查看字节码的代码如下

In [ ]:
# 查看字节码
import dis

def add_int(a, b):
    return a + b

dis.dis(add_int)

  4           0 LOAD_FAST                0 (a)
              2 LOAD_FAST                1 (b)
              4 BINARY_ADD
              6 RETURN_VALUE
